# Improving Fine-tuned Model using RAG

Code authored by: Shaw Talebi <br>
Article link: https://towardsdatascience.com/how-to-improve-llms-with-rag-abdc132f76ac <br>
Video link: https://youtu.be/Ylz779Op9Pw?si=iOvBETQDrgoK_sO6 <br>
<br>
Colab: https://colab.research.google.com/drive/1peJukr-9E1zCo1iAalbgDPJmNMydvQms?usp=sharing

### imports

In [3]:
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q
!pip install peft -q
!pip install auto-gptq -q
!pip install optimum -q
!pip install bitsandbytes -q
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

cuda_is_available = torch.cuda.is_available()

### Define Settings

In [4]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 50


LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

In [5]:
# some ad hoc document refinement
documents = SimpleDirectoryReader("txt_data").load_data()
print(len(documents))
for doc in documents:
    if "시행한다" in doc.text:
        documents.remove(doc)
        continue

    if "삭제" in doc.text:
        documents.remove(doc)

    if " 부칙" in doc.text:
        documents.remove(doc)

print(len(documents))

2
1


In [6]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

### Set Up Search Function

In [7]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [8]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [9]:
# query documents
query = "복수전공으로 인증 안된 전공은 뭐가 있어?"
response = query_engine.query(query)

In [10]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
&lt;개정 2010. 1. 21&gt;
④ 캠퍼스간 복수전공은 “미래캠퍼스 전공에 관한 내규”를 따른다. &lt;개정 2019.09.26.&gt;
제4조 (신청 및 허가)
① 졸업예정자 복수전공을 지원하고자 하는 자는 지정된 기간 내에 지원서를 제출하여 총장의 허가를 받아야 한다. &lt;개정 2023. 2. 4.&gt;
② 서류심사와 구술시험을 실시함을 원칙으로 하되, 필요에 따라 필기시험을 과할 수도 있다.

제1조 (목적)
이 시행세칙은 학칙 제40조에 따라 복수전공제 시행에 필요한 사항을 규정함을 목적으로 한다.
제2조 (범위)
① 본 대학교의 모든 학과(전공)를 복수전공 학과로 개방함을 원칙으로 한다.다만 다음 각호의 1에 해당하는 학과로의 복수전공은 허용하지 않는다. &lt;개정 2020.8.27.&gt;
1. 의학과, 치의학과, 간호학과, 약학과
2. 건축학(5년제), 음악대학 각 학과 &lt;개정 2023. 2. 4.&gt;
3.

건축학(5년제), 음악대학 각 학과 &lt;개정 2023. 2. 4.&gt;
3. 언더우드국제대학 및 글로벌인재대학 내 각 학부 및 전공. 단 해당 대학 소속 학생의 복수전공은 가능함
4. 물리치료학과, 작업치료학과, 임상병리학과, 방사선학과, 보건과학부(특별학사학위과정)
5. 학칙 제88조의4에 의한 계약학과
② 학과(학부)의 수용능력 등을 감안하여 복수전공자의 수를 제한할 수 있다.




### Import LLM

In [27]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

korean_model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
korean_model = AutoModelForCausalLM.from_pretrained(
    korean_model_name,
    device_map="auto",  
    trust_remote_code=False,  
    revision="main"  
)

tokenizer = AutoTokenizer.from_pretrained(korean_model_name, use_fast=True)
print(f"Model and Tokenizer loaded successfully for {korean_model_name}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and Tokenizer loaded successfully for Bllossom/llama-3.2-Korean-Bllossom-3B


### Use LLM

## this is the korean LLM without RAG

In [28]:
# prompt (no context)
intstructions_string = f"""너의 이름은 GLCPT다, 지금은 연세대학교 글로벌인재대학에 있는 학사지도교수이다. 학생들은 세계에 여러 국가에서 온 학생이고, 최대적으로 친적하게 대답\
해주고, 정보는 완벽하게 맞아야 된다. 모르는 정보는 그냥 모른다고 하면 됩니다.마음데로 대답하면 절대 안되

대답은 다음 양식으로 출력하세요:
1.출처
2.물움에 대한 대답
[end]
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''
korean_comment = "복수전공으로 인증 안된 전공은 뭐가 있어?"

prompt = prompt_template(korean_comment)
korean_model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = korean_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>[INST] 너의 이름은 GLCPT다, 지금은 연세대학교 글로벌인재대학에 있는 학사지도교수이다. 학생들은 세계에 여러 국가에서 온 학생이고, 최대적으로 친적하게 대답해주고, 정보는 완벽하게 맞아야 된다. 모르는 정보는 그냥 모른다고 하면 됩니다.마음데로 대답하면 절대 안되

대답은 다음 양식으로 출력하세요:
1.출처
2.물움에 대한 대답
[end]
 
복수전공으로 인증 안된 전공은 뭐가 있어? 
[/INST] 

1.출처
2.물움에 대한 대답
[end]

출처:
https://www.glovis.kr/contents/120001/120002/120003/

물움에 대한 대답:
물은 물론 물질의 기본적인 성질을 가진 물체로, 다양한 형태와 크기에서 존재하며, 지구의 생명체와 생태계에 필수적인 역할을 하고 있다. 물은 물리적, 화학적, 생물학적 관점에서 다양한 분야에서 연구되고 있다. 물의 특성, 물질의 물리학적, 화학적, 생물학적 특성, 물질의 물리학적, 화학적, 생물학적 특성 등 다양한 주제에서 연구되고 있다. 물은 지구의 생명체와 생태계에 필수적인 역할을 하고 있으며, 다양한 생태계에서 물의 역할을 연구하고 있다. 물의 역할은 식물의 물을 흡수하는 역할, 동물의 물을 흡수하는 역할, 지구의 기후를 형성하는 역할 등으로 나뉘며, 각 생태계에서 물의 역할이 매우 중요하다. 물의 역할은 식물의 물을 흡수하는 역할


## this is the korean LLM with RAG

In [29]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]너의 이름은 GLCPT다, 지금은 연세대학교 글로벌인재대학에 있는 학사지도교수이다. 학생들은 세계에 여러 국가에서 온 학생이고, 최대적으로 친적하게 대답\
해주고, 정보는 완벽하게 맞아야 된다. 모르는 정보는 그냥 모른다고 하면 됩니다.마음데로 대답하면 절대 안되.

대답은 다음 양식으로 출력하세요:
1.출처
2.물움에 대한 대답
{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""
korean_comment = "복수전공으로 인증 안된 전공은 뭐가 있어?"
prompt = prompt_template_w_context(context, korean_comment)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = korean_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>[INST]너의 이름은 GLCPT다, 지금은 연세대학교 글로벌인재대학에 있는 학사지도교수이다. 학생들은 세계에 여러 국가에서 온 학생이고, 최대적으로 친적하게 대답해주고, 정보는 완벽하게 맞아야 된다. 모르는 정보는 그냥 모른다고 하면 됩니다.마음데로 대답하면 절대 안되.

대답은 다음 양식으로 출력하세요:
1.출처
2.물움에 대한 대답
Context:
&lt;개정 2010. 1. 21&gt;
④ 캠퍼스간 복수전공은 “미래캠퍼스 전공에 관한 내규”를 따른다. &lt;개정 2019.09.26.&gt;
제4조 (신청 및 허가)
① 졸업예정자 복수전공을 지원하고자 하는 자는 지정된 기간 내에 지원서를 제출하여 총장의 허가를 받아야 한다. &lt;개정 2023. 2. 4.&gt;
② 서류심사와 구술시험을 실시함을 원칙으로 하되, 필요에 따라 필기시험을 과할 수도 있다.

제1조 (목적)
이 시행세칙은 학칙 제40조에 따라 복수전공제 시행에 필요한 사항을 규정함을 목적으로 한다.
제2조 (범위)
① 본 대학교의 모든 학과(전공)를 복수전공 학과로 개방함을 원칙으로 한다.다만 다음 각호의 1에 해당하는 학과로의 복수전공은 허용하지 않는다. &lt;개정 2020.8.27.&gt;
1. 의학과, 치의학과, 간호학과, 약학과
2. 건축학(5년제), 음악대학 각 학과 &lt;개정 2023. 2. 4.&gt;
3.

건축학(5년제), 음악대학 각 학과 &lt;개정 2023. 2. 4.&gt;
3. 언더우드국제대학 및 글로벌인재대학 내 각 학부 및 전공. 단 해당 대학 소속 학생의 복수전공은 가능함
4. 물리치료학과, 작업치료학과, 임상병리학과, 방사선학과, 보건과학부(특별학사학위과정)
5. 학칙 제88조의4에 의한 계약학과
② 학과(학부)의 수용능력 등을 감안하여 복수전공자의 수를 제한할 수 있다.


Please respond to the following comment. Use the context above if it is helpful.

복

## English version model

In [21]:
eng_model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
eng_model = AutoModelForCausalLM.from_pretrained(model_name,
                        device_map="auto",
                        trust_remote_code=False,
                        revision="main")

config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
eng_model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(eng_model_name, use_fast=True)

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

## this is the English llm with RAG

In [24]:
eng_comment = "what major cannot being the target of the double major?"
prompt_template_w_context = lambda context, comment: f"""[INST]GLC학생 도움이, functioning as a virtual consultant, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–GLC학생 도움이'. \
GLC학생 도움이 will tailor the length of its responses to match the question, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

{context}

Please respond to the following comment. Use the context above if it is helpful.
and answer in a format:
1.source
2.answer
do not answer more than the answer, and **글로벌인재대학** is Global Leaders College, **언더우드국제대학** is Underwood International College
**generate the answer whole in english**
{eng_comment}

[/INST]
"""
prompt = prompt_template_w_context(context, eng_comment)
inputs = tokenizer(prompt, return_tensors="pt")
outputs = eng_model.generate(input_ids=inputs["input_ids"].to("cuda") if cuda_is_available else inputs["input_ids"].to("cpu"), max_new_tokens=280)

decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[INST]GLC학생 도움이, functioning as a virtual consultant, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–GLC학생 도움이'. GLC학생 도움이 will tailor the length of its responses to match the question, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Context:
&lt;개정 2010. 1. 21&gt;
④ 캠퍼스간 복수전공은 “미래캠퍼스 전공에 관한 내규”를 따른다. &lt;개정 2019.09.26.&gt;
제4조 (신청 및 허가)
① 졸업예정자 복수전공을 지원하고자 하는 자는 지정된 기간 내에 지원서를 제출하여 총장의 허가를 받아야 한다. &lt;개정 2023. 2. 4.&gt;
② 서류심사와 구술시험을 실시함을 원칙으로 하되, 필요에 따라 필기시험을 과할 수도 있다.

제1조 (목적)
이 시행세칙은 학칙 제40조에 따라 복수전공제 시행에 필요한 사항을 규정함을 목적으로 한다.
제2조 (범위)
① 본 대학교의 모든 학과(전공)를 복수전공 학과로 개방함을 원칙으로 한다.다만 다음 각호의 1에 해당하는 학과로의 복수전공은 허용하지 않는다. &lt;개정 2020.8.27.&gt;
1. 의학과, 치의학과, 간호학과, 약학과
2. 건축학(5년제), 음악대학 각 학과 &lt;개정 2023. 2. 4.&gt;
3.

건축학(5년제), 음악대학 각 학과 &lt;개정 2023. 2. 4.&gt;
3. 언더우드국제대학 및 글로벌인재대학 내 각 학부 및 전공. 단 해당